###  Imports

In [1]:
import torch

import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

from dataset import FiftyOneTorchDataset
from model import create_model
from utils import add_detections, get_transforms

import config

torch.manual_seed(1)

In [2]:
print(fo.list_datasets())

[]


In [3]:
if len(fo.list_datasets()) > 0:
    dataset = fo.load_dataset("coco-2017-validation")
    dataset.delete()

In [4]:
#Load in the dataset from the FiftyOne model Zoo
fo_dataset = foz.load_zoo_dataset("coco-2017", "validation")

#needed to calculate image height and width
fo_dataset.compute_metadata()

session = fo.launch_app(fo_dataset)

subset = False

train_transforms, test_transforms = get_transforms()

if subset:
    # to filter certain items from the dataset we can
    item_list = ["car", "dog", "bus", 'fork', 'tie', 'person', 'airplane']
    item_list = ["man", "boy", "skis"]
    #item_list = ['airplane', 'cat']
    item_view = fo_dataset.filter_labels("ground_truth",
            F("label").is_in(item_list))

    #session.view = item_view

    # split the dataset in train and test set
    train_view = item_view.take((len(item_view) * config.TRAIN_TEST_SPLIT), seed=51)
    test_view = item_view.exclude([s.id for s in train_view])

else:
    train_view = fo_dataset.take(len(fo_dataset) * config.TRAIN_TEST_SPLIT)
    test_view = fo_dataset.exclude([s.id for s in train_view])
    item_list = fo_dataset.distinct("ground_truth.detections.label")

print(f'Traning on {len(train_view)} samples')
print(f'Testing on {len(test_view)} samples')

# use our dataset and defined transformations
train_dataset = FiftyOneTorchDataset(train_view, train_transforms,
        classes=item_list)
evaluation_dataset = FiftyOneTorchDataset(test_view, test_transforms,
        classes=item_list)

# #this is needed for later use, but not for creating the dataset
# if item_list[0] != 'background':
#      item_list.insert(0,'background')

Found annotations at 'C:\Users\blain\fiftyone\coco-2017\raw\instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading 'coco-2017' split 'validation'
 100% |███████████████| 5000/5000 [14.5s elapsed, 0s remaining, 358.6 samples/s]      
Dataset 'coco-2017-validation' created


Traning on 4000 samples
Testing on 1000 samples


# Check Faster RCNN performance

In [5]:
# MODEL_TYPE = 'CLIP-Backbone-FRCNN'
# CHECKPOINT_NAME = f'{MODEL_TYPE}_epoch_28.pth'
#
# if item_list[0] != 'background':
#      item_list.insert(0,'background')
#
# frcnn_model = create_model(MODEL_TYPE, classes=item_list)
# checkpoint = torch.load(CHECKPOINT_NAME)
# frcnn_model = create_model(MODEL_TYPE, classes=item_list)
#
# frcnn_model.load_state_dict(checkpoint)
# frcnn_model.eval()
#
# add_detections(frcnn_model, evaluation_dataset, fo_dataset, field_name="frcnn_predictions")
#
# results = fo.evaluate_detections(
#     test_view,
#     "frcnn_predictions",
#     classes=item_list,
#     eval_key="eval",
#     compute_mAP=True
# )
# session.view = item_view
# print(f'mAP: {results.mAP()}')
# results.print_report()

# Check CLIP FRCNN performance

In [6]:
# test out the trained CLIP-FRCNN
MODEL_TYPE = 'CLIP-RPN'
WEIGHTS_NAME = 'rpn'




# tokenize item list for CLIP
import clip
_, preprocess = clip.load("RN50", device=config.DEVICE)

if item_list[0] != '':
     item_list.insert(0,' ')

text_tokens = clip.tokenize(["This is " + desc for desc in item_list]).cuda()

CHECKPOINT_NAME = f'{MODEL_TYPE}_{WEIGHTS_NAME}.pth'
checkpoint = torch.load(CHECKPOINT_NAME)
clip_frcnn_model = create_model(MODEL_TYPE, classes=text_tokens)

clip_frcnn_model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
print(f'loaded checkpoint at epoch {epoch}')

clip_frcnn_model.eval()


add_detections(clip_frcnn_model, evaluation_dataset, fo_dataset, field_name="clip_RPN_predictions")

results = fo.evaluate_detections(
    test_view,
    "clip_RPN_predictions",
    classes=item_list,
    eval_key="clip_eval",
    compute_mAP=True
)

print(f'mAP: {results.mAP()}')
results.print_report()

session.view = test_view

loaded checkpoint at epoch 23
Using device cuda
   0% |/--------------|    1/1000 [226.1ms elapsed, 3.8m remaining, 4.4 samples/s] 

C:\Users\blain\anaconda3\envs\torch-frcnn\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


 100% |███████████████| 1000/1000 [2.3m elapsed, 0s remaining, 6.7 samples/s]      
Evaluating detections...
 100% |███████████████| 1000/1000 [6.3s elapsed, 0s remaining, 148.8 samples/s]      
Performing IoU sweep...
 100% |███████████████| 1000/1000 [9.5s elapsed, 0s remaining, 98.4 samples/s]       
mAP: 0.012400741353554522
                precision    recall  f1-score   support

                     0.00      0.00      0.00         0
      airplane       0.02      0.30      0.04        23
         apple       0.03      0.03      0.03        38
      backpack       0.05      0.04      0.05        67
        banana       0.00      0.00      0.00        37
  baseball bat       0.00      0.00      0.00        20
baseball glove       0.04      0.05      0.04        22
          bear       0.13      0.21      0.16        14
           bed       0.11      0.32      0.16        31
         bench       0.04      0.12      0.06        74
       bicycle       0.06      0.06      0.06       